# 업무추진비 보고서

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import re, os
import category_kakao as kakao

In [2]:
file = 'output_data'
files = [i for i in os.listdir(file) if i.endswith('.csv')]
df = pd.read_csv(f'output_data\\{files[0]}')
df = df.drop(columns=['Unnamed: 0'])

In [3]:
df

,department,name,cost,people,date,category
0,감사위원회,석교리,93000,6,2023-03-02,한식
1,감사위원회,두툼,92000,5,2023-03-06,한식
2,감사위원회,궁궐,126000,6,2023-03-09,한식
3,감사위원회,나루스시,104000,5,2023-03-13,일식
4,감사위원회,아호정,107900,6,2023-03-21,중식
...,...,...,...,...,...,...
740,환경정책과,오롯이,78000,4,2023-03-22,일식
741,환경정책과,우리홍콩딤섬,79000,7,2023-03-29,중식
742,환경정책과,우나기칸,60000,4,2023-03-30,일식
743,회계과,오롯이,145800,8,2023-03-10,일식


## 업무 추진비 사용 경향

In [4]:
# 업무 추진비 사용 금액
department_cost = df.groupby('department')['cost'].sum()
print(department_cost)

department
감사위원회      964400
감염병관리과     417000
건설교통국     1271250
건축과        573000
경제산업국      947500
           ...   
한솔동        812500
해밀동        463000
환경녹지국     1095600
환경정책과      619200
회계과        314300
Name: cost, Length: 100, dtype: int64


In [5]:
# 부서별 인원수 대비 금액
department_people = df.groupby('department')['people'].sum()
department_cost_per_person = department_cost / department_people
print(round(department_cost_per_person), 1)

department
감사위원회     20965.0
감염병관리과    14893.0
건설교통국     13970.0
건축과       13643.0
경제산업국     24295.0
           ...   
한솔동       13771.0
해밀동       14030.0
환경녹지국     14805.0
환경정책과     15480.0
회계과       20953.0
Length: 100, dtype: float64 1


In [6]:
# 가장 많이 방문한 가게
name_count = df['name'].value_counts()
print(name_count)

오롯이         21
궁궐          13
사키          12
함흥면옥        10
내토왕코다리      10
            ..
로흐뚜          1
피크닉백         1
세종청사용포맛집     1
아비꼬          1
우리홍콩딤섬       1
Name: name, Length: 440, dtype: int64


In [7]:
# 가장 많은 금액을 쓴 가게
name_cost = df.groupby('name')['cost'].sum()
name_cost = name_cost.sort_values(ascending=False)
print(name_cost)

name
오롯이                1942900
궁궐                 1408700
함흥면옥               1008800
사키                  994400
낙지촌                 956200
                    ...   
선식당                  19100
온버터                  19000
폴바셋여의도국회대로점(서울)      15600
아크                   13600
아이갓에브리씽              10500
Name: cost, Length: 440, dtype: int64


In [8]:
# 이름별 비용 합계 계산
name_counts = name_count[name_count >= 2]
cost_count = name_cost/name_counts
cost_count = cost_count.dropna()
cost_count = cost_count.sort_values(ascending=False)

In [9]:
# 다양한 부서가 방문한 가게
depart_name = df.groupby('name')['department'].nunique()
depart_name = depart_name.sort_values(ascending=False)

### Markdown 문서 생성

In [10]:
header = f"""
# 세종시 업무추진비 맛집 ({files[0]})

업무추진비 URL : https://www.sejong.go.kr/bbs/R0091/list.do
"""

body = f"""
## 업무 추진비 사용 내역

### (1) 많이 방문한 가게

{name_count.head(5).to_markdown()}

### (2) 많은 금액을 지출한 가게

{name_cost.map('{:,.0f}'.format).head(5).to_markdown()}

### (3) 방문 횟수 대비 비용이 큰 가게

{cost_count.map('{:,.0f}'.format).head(5).to_markdown()}

### (4) 다양한 부서가 방문하는 가게

{depart_name.map('{:,.0f}'.format).head(5).to_markdown()}

"""

md_file = header + body

### MD 파일 작성

In [11]:
# 파일 작성
if not os.path.exists('report'):
    os.makedirs('report')

with open(f"report\\{files[0][:-4]}.md", "w", encoding="utf-8") as f:
  f.write(md_file)